# libs, dataset

In [ ]:
!rm -rf used_cars

In [5]:
!git lfs clone https://huggingface.co/datasets/imbalet/used_cars

          with new flags from `git clone`

`git clone` has been updated in upstream Git to have comparable
speeds to `git lfs clone`.
Клонирование в «used_cars»...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14 (delta 2), reused 0 (delta 0), pack-reused 4 (from 1)
Распаковка объектов: 100% (14/14), 3.30 КиБ | 1.65 МиБ/с, готово.


In [ ]:
!pip install numpy pandas scikit-learn tensorflow

# import

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

2024-12-01 17:24:07.001306: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733063047.676593  421892 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733063047.834377  421892 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 17:24:09.714081: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Creating db
in progress...

In [14]:
df = pd.read_csv("used_cars/dataset.csv")

In [ ]:
vars = ["mark", "model", "super_gen"]
v = ["engine", "transmission"]

In [ ]:
for i in set(df["model"]):
    if len(set(df[df["model"] == i]["mark"])) > 1: print(i, set(df[df["model"] == i]["mark"]))

In [15]:
data = {}
for index, row in df.iterrows():
    mark = row["mark"]
    model = row["model"]
    gen = row["super_gen"]
    tr = row["transmission"]
    en = row["engine"]
    if mark not in data:
        data[mark] = {}
    if model not in data[mark]:
        data[mark][model] = {}
    if gen not in data[mark][model]:
        data[mark][model][gen] = {"transmission": set(), "engine": set(), "year": set(), "body_type": set()}
    data[mark][model][gen]["transmission"].add({"name": str(tr), "text": str(tr)})
    data[mark][model][gen]["engine"].add({"name": str(en), "text": str(en)})
    data[mark][model][gen]["year"].add({"name": row["year"], "text": row["year"]})
    data[mark][model][gen]["body_type"].add({"name": row["body_type"], "text": row["body_type"]})



KeyboardInterrupt: 

In [ ]:
for i in data:
    for j in data[i]:
        for k in data[i][j]:
            data[i][j][k]["transmission"] = list(data[i][j][k]["transmission"])
            data[i][j][k]["engine"] = list(data[i][j][k]["engine"])
            data[i][j][k]["year"] = list(data[i][j][k]["year"])
            data[i][j][k]["body_type"] = list(data[i][j][k]["body_type"])

        print(data[i][j][k])

{'transmission': ['AUTOMATIC', 'VARIATOR'], 'engine': ['GASOLINE', 'HYBRID'], 'year': [2024, 2021, 2022, 2023], 'body_type': ['ALLROAD_5_DOORS']}
{'transmission': ['ROBOT'], 'engine': ['GASOLINE'], 'year': [2020, 2021, 2022], 'body_type': ['ALLROAD_5_DOORS']}
{'transmission': ['MECHANICAL', 'AUTOMATIC'], 'engine': ['GASOLINE'], 'year': [2000, 2001, 2002, 2003], 'body_type': ['SEDAN']}
{'transmission': ['AUTOMATIC'], 'engine': ['GASOLINE'], 'year': [2021, 2022, 2023], 'body_type': ['ALLROAD_5_DOORS']}
{'transmission': ['ROBOT'], 'engine': ['GASOLINE'], 'year': [2016, 2017], 'body_type': ['COUPE']}
{'transmission': ['VARIATOR'], 'engine': ['GASOLINE'], 'year': [2024], 'body_type': ['ALLROAD_5_DOORS']}
{'transmission': ['MECHANICAL', 'AUTOMATIC'], 'engine': ['GASOLINE', 'DIESEL'], 'year': [2002, 2003, 2004, 2005, 2006], 'body_type': ['SEDAN', 'HATCHBACK_3_DOORS', 'HATCHBACK_5_DOORS']}
{'transmission': ['AUTOMATIC', 'VARIATOR'], 'engine': ['GASOLINE'], 'year': [2003, 2004, 2005], 'body_typ

In [ ]:
for i in data:
    for j in data[i]:
        for k in data[i][j]:
            min_y = min( data[i][j][k]["year"])
            max_y = max( data[i][j][k]["year"])
            data[i][j][k]["year"] = [i for i in range(min_y, max_y + 1)]

In [16]:
cols = ["owners", 'region', 'steering_wheel', 'gear_type', "color"]
vars = {i: list(map(str, set(df[i]))) for i in cols}

In [11]:
vars["mark"] = data

In [ ]:
import json
with open("variants_cars.json", 'w') as f:
    json.dump(data, f, ensure_ascii = False)
    

In [13]:
import json
with open("variants.json", 'w') as f:
    json.dump(vars, f, ensure_ascii = False)

# dataset


In [2]:
df = pd.read_csv("used_cars/dataset.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'used_cars/dataset.csv'

In [ ]:
df.sample(3)

In [ ]:
df = df.drop(["doors", "class"], axis = 1)

In [ ]:
# df["doors"] = df["doors"].astype("object")
df["super_gen"] = df["super_gen"].astype("object")
df["year"] = df["year"].astype("object")
df["owners"] = df["owners"].astype("object")

In [ ]:
num_features = ["owners", "mileage", "power", "displacemnt"]
cat_features = df.select_dtypes(include=['object']).columns

In [ ]:
price = ["price"]

price_scaler = StandardScaler()

df[price] = price_scaler.fit_transform(df[price])

In [ ]:
scaler = StandardScaler()
df[num_features] = scaler.fit_transform(df[num_features])

In [ ]:
label_encoders = {}
for feature in cat_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le


In [ ]:
# creating config scale/encoders file
import json
scalers = {scaler.feature_names_in_[i]: (float(scaler.mean_[i]), float(scaler.scale_[i])) for i in range(len(scaler.mean_))}
scalers["price"] = (float(price_scaler.mean_[0]), float(price_scaler.scale_[0]))

labels = {i: list(map(str, label_encoders[i].classes_)) for i in label_encoders}

with open("configs.json", 'w', encoding="utf-8") as f:
    json.dump({"scalers": scalers, "labels": labels}, f, ensure_ascii=False, indent=4)

In [ ]:
df.sample(3)

In [ ]:
X = df.drop(columns=['price'])
y = df['price']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# model


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

In [ ]:
model.compile(optimizer='adam', loss = "mean_squared_error")

In [ ]:
model.fit(X_train, y_train, epochs=50, validation_split=0.1, batch_size=64)

In [ ]:
model.save('model.keras')

In [ ]:
test_loss = model.evaluate(X_test, y_test)
print(f'Test loss (MSE): {test_loss}')

In [ ]:
original_data = price_scaler.inverse_transform(np.array(y_test).reshape(-1, 1))

In [ ]:
pred_unscaled = price_scaler.inverse_transform(model.predict(X_test))

In [ ]:
# MAE
sum(abs(original_data - pred_unscaled)) / original_data.shape[0]